# Inter-Rater Agreement
#### Measure the agreement among pairs of raters of the HFC dataset (focusing only on 5/12 raters to simplify the analysis).

In [1]:
from typing import Optional, List

import numpy as np
import pandas as pd
import tqdm

import peyes
import analysis.utils as u
from analysis._article_results.hfc._helpers import *

GT_LABELERS = [GT1, GT2, GT3, GT4, GT5]

In [2]:
dataset = peyes.datasets.hfc(directory=u.DATASETS_DIR, save=False, verbose=True)

In [3]:
def calc_sample_level_agreement(
        dataframe: pd.DataFrame,
        metrics: List[str] = None,
        gt_labelers: List[str] = GT_LABELERS,
        pos_labels: Optional[np.ndarray] = None
) -> pd.DataFrame:
    metrics = metrics or ["balanced_accuracy", "cohen's_kappa", "mcc", "complement_nld"]
    trial_ids = dataframe[peyes.constants.TRIAL_ID_STR].unique()
    results = {}
    for i, trial_id in tqdm.tqdm(enumerate(trial_ids), total=len(trial_ids)):
        trial_data = dataframe[dataframe["trial_id"] == trial_id]
        labels_per_labeler = {lblr: trial_data[lblr].values for lblr in gt_labelers}
        for i, lblr1 in enumerate(labels_per_labeler.keys()):
            for j, lblr2 in enumerate(labels_per_labeler.keys()):
                if i >= j:
                    continue
                res = peyes.sample_metrics.calculate(labels_per_labeler[lblr1], labels_per_labeler[lblr2], *metrics, pos_labels=pos_labels)
                results[(trial_id, lblr1, lblr2)] = res
    results = pd.DataFrame(results).T
    results.index.names = [peyes.constants.TRIAL_ID_STR, u.GT_STR, u.PRED_STR]
    return results

### Entire Dataset
#### (A) All Annotators

In [10]:
overall_agreement = calc_sample_level_agreement(dataset)
overall_agreement_summary = overall_agreement.groupby(level=[u.GT_STR, u.PRED_STR]).describe().stack(0, future_stack=True)
overall_agreement_summary.index.names = [u.GT_STR, u.PRED_STR, "metric"]

overall_agreement_summary

 17%|█▋        | 12/70 [00:24<01:16,  1.31s/it]C:\Users\jonathanni\PycharmProjects\pEYES\venv\Lib\site-packages\sklearn\metrics\_classification.py:730: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
C:\Users\jonathanni\PycharmProjects\pEYES\venv\Lib\site-packages\sklearn\metrics\_classification.py:730: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
C:\Users\jonathanni\PycharmProjects\pEYES\venv\Lib\site-packages\sklearn\metrics\_classification.py:730: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
100%|██████████| 70/70 [00:47<00:00,  1.47it/s]


count      mean       std       min       25%  \
gt pred metric                                                             
DN JV   balanced_accuracy   70.0  0.954145  0.033710  0.782539  0.945949   
        cohen's_kappa       70.0  0.826188  0.129610  0.000000  0.785086   
        mcc                 70.0  0.835474  0.126290  0.000000  0.803230   
        complement_nld      70.0  0.954126  0.025022  0.880498  0.942743   
   MN   balanced_accuracy   70.0  0.957476  0.030907  0.803571  0.947174   
        cohen's_kappa       69.0  0.864698  0.090459  0.404600  0.828217   
        mcc                 70.0  0.858964  0.131277  0.000000  0.833054   
        complement_nld      70.0  0.963158  0.025797  0.846132  0.951135   
   RA   balanced_accuracy   70.0  0.951557  0.047473  0.728719  0.942947   
        cohen's_kappa       69.0  0.841203  0.124180  0.286615  0.815378   
        mcc                 70.0  0.838734  0.150050  0.000000  0.817043   
        complement_nld      70.0  0.954542  0.043200  0.760563  0.951396   
IH DN   balanced_accuracy   70.0  0.952527  0.065126  0.500000  0.946946   
        cohen's_kappa       70.0  0.856607  0.133165  0.000000  0.800699   
        mcc                 70.0  0.861905  0.129886  0.000000  0.817095   
        complement_nld      70.0  0.968320  0.025204  0.808415  0.961966   
   JV   balanced_accuracy   70.0  0.940821  0.039955  0.736074  0.929160   
        cohen's_kappa       70.0  0.768684  0.124463  0.484484  0.654667   
        mcc                 70.0  0.788565  0.109197  0.507681  0.695661   
        complement_nld      70.0  0.935714  0.038319  0.768791  0.920471   
   MN   balanced_accuracy   70.0  0.940211  0.066872  0.500000  0.937731   
        cohen's_kappa       70.0  0.785298  0.155654  0.000000  0.718101   
        mcc                 70.0  0.801098  0.144768  0.000000  0.745408   
        complement_nld      70.0  0.946770  0.036683  0.779412  0.931527   
   RA   balanced_accuracy   70.0  0.936205  0.073425  0.500000  0.929017   
        cohen's_kappa       70.0  0.765999  0.169422  0.000000  0.691394   
        mcc                 70.0  0.785083  0.156384  0.000000  0.718559   
        complement_nld      70.0  0.938571  0.047379  0.739022  0.931510   
JV MN   balanced_accuracy   70.0  0.930008  0.067022  0.500000  0.913367   
        cohen's_kappa       70.0  0.872403  0.141078  0.000000  0.854058   
        mcc                 70.0  0.876826  0.134930  0.000000  0.857048   
        complement_nld      70.0  0.966206  0.028421  0.832480  0.957386   
   RA   balanced_accuracy   70.0  0.921751  0.073219  0.500000  0.904699   
        cohen's_kappa       70.0  0.844764  0.159539  0.000000  0.829196   
        mcc                 70.0  0.850913  0.151512  0.000000  0.833857   
        complement_nld      70.0  0.956483  0.043286  0.768848  0.948507   
RA MN   balanced_accuracy   70.0  0.940937  0.064205  0.583651  0.929881   
        cohen's_kappa       69.0  0.883802  0.118583  0.184790  0.861581   
        mcc                 70.0  0.874839  0.154055  0.000000  0.865653   
        complement_nld      70.0  0.965864  0.044348  0.730737  0.968005   

                                50%       75%       max  
gt pred metric                                           
DN JV   balanced_accuracy  0.963407  0.971856  0.999172  
        cohen's_kappa      0.847105  0.903284  0.993013  
        mcc                0.850103  0.905752  0.993038  
        complement_nld     0.957655  0.968419  0.999172  
   MN   balanced_accuracy  0.962382  0.975582  1.000000  
        cohen's_kappa      0.877572  0.924482  0.992945  
        mcc                0.881153  0.924973  0.992969  
        complement_nld     0.967370  0.979010  1.000000  
   RA   balanced_accuracy  0.968134  0.979433  1.000000  
        cohen's_kappa      0.856382  0.921569  0.993016  
        mcc                0.862007  0.919050  0.993040  
        complement_nld     0.963255  0.977818  1.000000  
IH DN   balanced_accur

#### mean agreement over subset of GT annotators
GT annotators = ["IH", "DN", "JV", "MN", "RA"]

In [11]:
overall_agreement_mean = overall_agreement_summary[["mean", "std", "min", "50%", "max"]].groupby(level="metric").mean()
overall_agreement_mean

,mean,std,min,50%,max
metric,,,,,
balanced_accuracy,0.942564,0.056191,0.613455,0.957421,0.997823
cohen's_kappa,0.830965,0.134615,0.136049,0.854887,0.990621
complement_nld,0.954975,0.035766,0.791490,0.961927,0.999586
mcc,0.837240,0.138835,0.050768,0.860924,0.990658


#### (B) GT1-GT2 Agreement

In [5]:
gt1_gt2_overall_agreement = calc_sample_level_agreement(dataset, gt_labelers=[GT1, GT2])
gt1_gt2_overall_agreement_summary = gt1_gt2_overall_agreement.groupby(level=[u.GT_STR, u.PRED_STR]).describe().stack(0, future_stack=True)
gt1_gt2_overall_agreement_summary.index.names = [u.GT_STR, u.PRED_STR, "metric"]

gt1_gt2_overall_agreement_summary

100%|██████████| 70/70 [00:04<00:00, 14.13it/s]


count      mean       std       min       25%  \
gt pred metric                                                             
IH DN   balanced_accuracy   70.0  0.952527  0.065126  0.500000  0.946946   
        cohen's_kappa       70.0  0.856607  0.133165  0.000000  0.800699   
        mcc                 70.0  0.861905  0.129886  0.000000  0.817095   
        complement_nld      70.0  0.968320  0.025204  0.808415  0.961966   

                                50%       75%  max  
gt pred metric                                      
IH DN   balanced_accuracy  0.965825  0.983680  1.0  
        cohen's_kappa      0.885860  0.934016  1.0  
        mcc                0.887913  0.934514  1.0  
        complement_nld     0.971785  0.980753  1.0

### Free-Viewing Subset
#### (A) All Annotators

In [6]:
fv_dataset = dataset[dataset[peyes.constants.STIMULUS_TYPE_STR] == "free_viewing"]

fv_agreement = calc_sample_level_agreement(fv_dataset)
fv_agreement_summary = fv_agreement.groupby(level=[u.GT_STR, u.PRED_STR]).describe().stack(0, future_stack=True)
fv_agreement_summary.index.names = [u.GT_STR, u.PRED_STR, "metric"]

fv_agreement_mean = fv_agreement_summary[["mean", "std", "min", "50%", "max"]].groupby(level="metric").mean()
fv_agreement_mean

100%|██████████| 10/10 [00:23<00:00,  2.37s/it]


,mean,std,min,50%,max
metric,,,,,
balanced_accuracy,0.922851,0.048410,0.803203,0.937937,0.964052
cohen's_kappa,0.749121,0.112153,0.471965,0.781806,0.846246
complement_nld,0.940056,0.034906,0.850641,0.951405,0.966081
mcc,0.763777,0.104619,0.504275,0.795819,0.852501


#### (B) GT1-GT2 Agreement

In [7]:
gt1_gt2_fv_agreement = calc_sample_level_agreement(fv_dataset, gt_labelers=[GT1, GT2])
gt1_gt2_fv_agreement_summary = gt1_gt2_fv_agreement.groupby(level=[u.GT_STR, u.PRED_STR]).describe().stack(0, future_stack=True)
gt1_gt2_fv_agreement_summary.index.names = [u.GT_STR, u.PRED_STR, "metric"]

gt1_gt2_fv_agreement_summary

100%|██████████| 10/10 [00:02<00:00,  4.10it/s]


count      mean       std       min       25%  \
gt pred metric                                                             
IH DN   balanced_accuracy   10.0  0.944648  0.020104  0.900926  0.939128   
        cohen's_kappa       10.0  0.806712  0.042060  0.764313  0.777827   
        mcc                 10.0  0.813266  0.038667  0.773348  0.786875   
        complement_nld      10.0  0.964497  0.011655  0.935806  0.961901   

                                50%       75%       max  
gt pred metric                                           
IH DN   balanced_accuracy  0.945310  0.959501  0.970974  
        cohen's_kappa      0.781803  0.837265  0.891147  
        mcc                0.793781  0.839614  0.891203  
        complement_nld     0.966570  0.970491  0.978681

### (C) RA-MN Agreement
(Raters _RA_ and _MN_ are the ground truth raters for dataset _lund2013_, so lets check their agreement in this dataset too)

In [8]:
ra_mn_fv_agreement = calc_sample_level_agreement(fv_dataset, gt_labelers=["RA", "MN"])
ra_mn_fv_agreement_summary = ra_mn_fv_agreement.groupby(level=[u.GT_STR, u.PRED_STR]).describe().stack(0, future_stack=True)
ra_mn_fv_agreement_summary.index.names = [u.GT_STR, u.PRED_STR, "metric"]

ra_mn_fv_agreement_summary

100%|██████████| 10/10 [00:02<00:00,  4.12it/s]


count      mean       std       min       25%  \
gt pred metric                                                             
RA MN   balanced_accuracy   10.0  0.880287  0.112928  0.583651  0.887296   
        cohen's_kappa       10.0  0.774451  0.218439  0.184790  0.796618   
        mcc                 10.0  0.776534  0.215308  0.194232  0.797772   
        complement_nld      10.0  0.940380  0.069153  0.751241  0.936417   

                                50%       75%       max  
gt pred metric                                           
RA MN   balanced_accuracy  0.914027  0.938763  0.973741  
        cohen's_kappa      0.847910  0.883487  0.927445  
        mcc                0.848371  0.883530  0.927873  
        complement_nld     0.968995  0.974736  0.981572